Prepare summary datasets.

In [1]:
import numpy as np
import pandas as pd
import pickle

from isttc.scripts.cfg_global import project_folder_path

In [2]:
dataset_folder = project_folder_path + 'synthetic_dataset\\'
results_folder = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau\\'

### Helpers

In [3]:
def calculate_acf_decline_flag(acf_, start_idx=1, end_idx=4):
    acf_decay = np.all(np.diff(acf_[start_idx:end_idx]) <= 0)
    return acf_decay

def get_tau_df(acf_dict_, signal_len_, method_, alphas_, fr_values_, taus_ms_, bin_size_=50):
    data = []
    for unit_id, unit_data in acf_dict_.items():
        taus = unit_data['taus']  
        data.append({
            'unit_id': unit_id,
            'tau': taus['tau'],
            'tau_lower': taus['tau_lower'],
            'tau_upper': taus['tau_upper'],
            'fit_r_squared': taus['fit_r_squared'],
            'acf_decline': calculate_acf_decline_flag(unit_data['acf'], start_idx=1, end_idx=4)
        })
    tau_df = pd.DataFrame(data)
    tau_df['method'] = method_
    tau_df['tau_ms'] = tau_df['tau'] * bin_size_
    tau_df['duration_s'] = signal_len_
    tau_df['fr'] = fr_values_
    tau_df['alpha'] = alphas_
    tau_df['tau_ms_true'] = taus_ms_
    tau_df['tau_diff_abs'] = np.abs(tau_df['tau_ms'] - tau_df['tau_ms_true'])
    tau_df['tau_diff_rel'] = tau_df['tau_diff_abs'] / tau_df['tau_ms_true'] * 100
    tau_df['ci_width'] = np.abs(tau_df['tau_upper'] - tau_df['tau_lower'])
    
    rows_with_nans_df = tau_df[tau_df.isna().any(axis=1)]
    n_rows_with_nan = len(rows_with_nans_df)
    print(f'N rows with NaNs {n_rows_with_nan}')
    
    return tau_df

def get_trials_plot_df(trial_dict_, n_trials_, method_, alphas_, fr_values_, taus_ms_, n_iteration_=None, bin_size_=50):
    records = []
    for unit_id, data in trial_dict_.items():
        taus = data['taus']
        acfs = data['acf']

        if n_iteration_ is not None:
            # only one trial per unit
            idx = n_iteration_[unit_id]
            taus_to_iter = [(taus[idx], acfs[idx])]
        else:
            # all trials for this unit
            taus_to_iter = zip(taus, acfs)

        for tau_dict, acf_array in taus_to_iter:
            records.append({
                'unit_id': unit_id,
                'tau': tau_dict['tau'],
                'tau_lower': tau_dict['tau_lower'],
                'tau_upper': tau_dict['tau_upper'],
                'fit_r_squared': tau_dict['fit_r_squared'],
                'acf_decline': calculate_acf_decline_flag(acf_array, start_idx=1, end_idx=4),
                'method': method_,
            })

    tau_df = pd.DataFrame.from_records(records)
    tau_df['tau_ms'] = tau_df['tau'] * bin_size_
    tau_df['n_trials'] = n_trials_
    tau_df['fr'] = fr_values_
    tau_df['alpha'] = alphas_
    tau_df['tau_ms_true'] = taus_ms_
    tau_df['tau_diff_abs'] = np.abs(tau_df['tau_ms'] - tau_df['tau_ms_true'])
    tau_df['tau_diff_rel'] = tau_df['tau_diff_abs'] / tau_df['tau_ms_true'] * 100
    tau_df['ci_width'] = np.abs(tau_df['tau_upper'] - tau_df['tau_lower'])

    nan_count = tau_df.isna().any(axis=1).sum()
    if nan_count > 0:
        print(f'N rows with NaNs {nan_count}')

    return tau_df

### Load spike trains

In [4]:
with open(dataset_folder + 'spike_trains.pkl', 'rb') as f:
    data = pickle.load(f)

spike_trains = data['spike_trains']
alphas = data['alphas']
fr_values = data['fr_values']
taus_ms = data['tau_ms']
duration_ms = data['duration_ms']

print(f'n spike trains {len(spike_trains)}, len {spike_trains[0][-1]/1000}, duration_ms {duration_ms}')

lv_df = pd.read_pickle(results_folder + 'lv_df.pkl')
print(f'Lv loaded for n spike trains {len(lv_df)}')

#fs = 1000

n spike trains 100000, len 598.8243581617338, duration_ms [600000 600000 600000 ... 600000 600000 600000]
Lv loaded for n spike trains 100000


### ACF, iSTTC, PersonR, iSTTC trails (сoncat)

The summary dataset includes only units where all 4 methods have valid (non-NaN) rows and fit_r_squared ≥ 0.

In [ ]:
# load per method
with open(results_folder + 'tau_isttc_full_50ms_20lags_dict.pkl', "rb") as f:
    isttc_full_dict = pickle.load(f)

with open(results_folder + 'tau_acf_full_50ms_20lags_dict.pkl', "rb") as f:
    acf_full_dict = pickle.load(f)

with open(results_folder + 'tau_pearsonr_trial_50ms_20lags_dict.pkl', "rb") as f:
    pearsonr_trial_avg_dict = pickle.load(f)

with open(results_folder + 'tau_isttc_trial_concat_50ms_20lags_dict.pkl', "rb") as f:
    sttc_trial_concat_dict = pickle.load(f)

print(f'len pearsonr_trial_avg_dict {len(pearsonr_trial_avg_dict)}')
print(f'len sttc_trial_concat_dict {len(sttc_trial_concat_dict)}')
print(f'len isttc_full_dict {len(isttc_full_dict)}')
print(f'len acf_full_dict {len(acf_full_dict)}')

acf_tau_full_df = get_tau_df(acf_full_dict, 600, 'acf_full', alphas, fr_values, taus_ms)
isttc_tau_full_df = get_tau_df(isttc_full_dict, 600, 'isttc_full', alphas, fr_values, taus_ms)
random_trials_impl = np.zeros(len(sttc_trial_concat_dict)).astype(int)
pearsontr_trial_avg_plot_df = get_trials_plot_df(pearsonr_trial_avg_dict, 40, 'pearsonr_trial_avg', alphas, fr_values, taus_ms, random_trials_impl)
sttc_trial_concat_plot_df = get_trials_plot_df(sttc_trial_concat_dict, 40, 'sttc_trial_concat', alphas, fr_values, taus_ms, random_trials_impl)

# prepare df
tau_all_long_df = pd.concat([acf_tau_full_df, isttc_tau_full_df, pearsontr_trial_avg_plot_df, sttc_trial_concat_plot_df])
tau_all_long_df.reset_index(inplace=True, drop=True)
tau_all_long_df.drop(columns=['duration_s'], inplace=True)
tau_all_long_df.drop(columns=['n_trials'], inplace=True)
tau_all_long_df = tau_all_long_df.merge(lv_df[['unit_id', 'lv']].copy(), on='unit_id', how='left')
print(f'len tau_all_long_df {len(tau_all_long_df)}')
# leave only units with both methods (no NaNs and r_squared >= 0)
required = {'acf_full', 'isttc_full', 'pearsonr_trial_avg', 'sttc_trial_concat'}
tau_all_long_df_clean = (
    tau_all_long_df.groupby("unit_id")
      .filter(lambda g: (
          len(g) == 4
          and set(g["method"]) == required
          and g.notna().all().all()
          and (g["fit_r_squared"] >= 0).all()
      ))
)
tau_all_long_df_clean.reset_index(inplace=True, drop=True)
print(f'len tau_all_long_df_clean {len(tau_all_long_df_clean)}, per method {len(tau_all_long_df_clean)/4}')

#save df
tau_all_long_df_clean.to_csv(results_folder + 'summary_tau_all_long_df.csv')
tau_all_long_df_clean.to_pickle(results_folder + 'summary_tau_all_long_df.pkl')

# save full df as well (with NaNs and negative r_squared)
tau_all_long_df.to_csv(results_folder + 'summary_tau_all_long_df_all_units.csv')
tau_all_long_df.to_pickle(results_folder + 'summary_tau_all_long_df_all_units.pkl')

### ACF vs iSTTC, full signal

The summary dataset includes only units where all 2 methods have valid (non-NaN) rows and fit_r_squared ≥ 0.

In [ ]:
# load per method
with open(results_folder + 'tau_isttc_full_50ms_20lags_dict.pkl', "rb") as f:
    isttc_full_dict = pickle.load(f)

with open(results_folder + 'tau_acf_full_50ms_20lags_dict.pkl', "rb") as f:
    acf_full_dict = pickle.load(f)

print(f'len isttc_full_dict {len(isttc_full_dict)}')
print(f'len acf_full_dict {len(acf_full_dict)}')

acf_tau_full_df = get_tau_df(acf_full_dict, 600, 'acf_full', alphas, fr_values, taus_ms)
isttc_tau_full_df = get_tau_df(isttc_full_dict, 600, 'isttc_full', alphas, fr_values, taus_ms)

# prepare df
tau_full_long_df = pd.concat([acf_tau_full_df, isttc_tau_full_df])
tau_full_long_df.reset_index(inplace=True, drop=True)
tau_full_long_df = tau_full_long_df.merge(lv_df[['unit_id', 'lv']].copy(), on='unit_id', how='left')
print(f'len tau_full_long_df {len(tau_full_long_df)}')
# leave only units with both methods (no NaNs and r_squared >= 0)
required = {"acf_full", "isttc_full"}
tau_full_long_df_clean = (
    tau_full_long_df.groupby("unit_id")
      .filter(lambda g: (
          len(g) == 2
          and set(g["method"]) == required
          and g.notna().all().all()
          and (g["fit_r_squared"] >= 0).all()
      ))
)
tau_full_long_df_clean.reset_index(inplace=True, drop=True)
print(f'len tau_full_long_df_clean {len(tau_full_long_df_clean)}, per method {len(tau_full_long_df_clean)/2}')

#save df
tau_full_long_df_clean.to_csv(results_folder + 'summary_tau_full_long_df.csv')
tau_full_long_df_clean.to_pickle(results_folder + 'summary_tau_full_long_df.pkl')

### PearsonR vs iSTTC (concat and avg), trails

The summary dataset includes only units where all 3 methods have valid (non-NaN) rows and fit_r_squared ≥ 0. STTC trail avg is not included in the paper figures. 

In [ ]:
# load per method
with open(results_folder + 'tau_pearsonr_trial_50ms_20lags_dict.pkl', "rb") as f:
    pearsonr_trial_avg_dict = pickle.load(f)

with open(results_folder + 'tau_sttc_trial_avg_50ms_20lags_dict.pkl', "rb") as f:
    sttc_trial_avg_dict = pickle.load(f)

with open(results_folder + 'tau_isttc_trial_concat_50ms_20lags_dict.pkl', "rb") as f:
    sttc_trial_concat_dict = pickle.load(f)

print(f'len pearsonr_trial_avg_dict {len(pearsonr_trial_avg_dict)}')
print(f'len sttc_trial_avg_dict {len(sttc_trial_avg_dict)}')
print(f'len sttc_trial_concat_dict {len(sttc_trial_concat_dict)}')

# for trial based measures one realization of trials is taken
random_trials_impl = np.zeros(len(sttc_trial_concat_dict)).astype(int)

pearsontr_trial_avg_plot_df = get_trials_plot_df(pearsonr_trial_avg_dict, 40, 'pearsonr_trial_avg', alphas, fr_values, taus_ms, random_trials_impl)
sttc_trial_concat_plot_df = get_trials_plot_df(sttc_trial_concat_dict, 40, 'sttc_trial_concat', alphas, fr_values, taus_ms, random_trials_impl)
sttc_trial_avg_plot_df = get_trials_plot_df(sttc_trial_avg_dict, 40, 'sttc_trial_avg', alphas, fr_values, taus_ms, random_trials_impl)

# prepare df
tau_trials_long_df = pd.concat([pearsontr_trial_avg_plot_df, sttc_trial_concat_plot_df, sttc_trial_avg_plot_df])
tau_trials_long_df.reset_index(inplace=True, drop=True)
tau_trials_long_df = tau_trials_long_df.merge(lv_df[['unit_id', 'lv']].copy(), on='unit_id', how='left')
print(f'len tau_trials_long_df {len(tau_trials_long_df)}')
# leave only units with both methods (no NaNs and r_squared >= 0)
required = {'pearsonr_trial_avg', 'sttc_trial_concat', 'sttc_trial_avg'}
tau_trials_long_df_clean = (
    tau_trials_long_df.groupby("unit_id")
      .filter(lambda g: (
          len(g) == 3
          and set(g["method"]) == required
          and g.notna().all().all()
          and (g["fit_r_squared"] >= 0).all()
      ))
)
tau_trials_long_df_clean.reset_index(inplace=True, drop=True)
print(f'len tau_trials_long_df_clean {len(tau_trials_long_df_clean)}, per method {len(tau_trials_long_df_clean)/3}')

#save df
tau_trials_long_df_clean.to_csv(results_folder + 'summary_tau_trials_long_df.csv')
tau_trials_long_df_clean.to_pickle(results_folder + 'summary_tau_trials_long_df.pkl')

### ACF vs iSTTC, varying signal length (60, 150, 300, 450, 600 seconds)

In [ ]:
# 60 sec
with open(results_folder + 'tau_acf_full_50ms_20lags_len_60sec_dict.pkl', "rb") as f:
    acf_60_dict = pickle.load(f)
with open(results_folder + 'tau_isttc_full_50ms_20lags_len_60sec_dict.pkl', "rb") as f:
    sttc_60_dict = pickle.load(f)

# 150 sec
with open(results_folder + 'tau_acf_full_50ms_20lags_len_150sec_dict.pkl', "rb") as f:
    acf_150_dict = pickle.load(f)
with open(results_folder + 'tau_isttc_full_50ms_20lags_len_150sec_dict.pkl', "rb") as f:
    sttc_150_dict = pickle.load(f)

# 300 sec
with open(results_folder + 'tau_acf_full_50ms_20lags_len_300sec_dict.pkl', "rb") as f:
    acf_300_dict = pickle.load(f)
with open(results_folder + 'tau_isttc_full_50ms_20lags_len_300sec_dict.pkl', "rb") as f:
    sttc_300_dict = pickle.load(f)

# 450 sec
with open(results_folder + 'tau_acf_full_50ms_20lags_len_450sec_dict.pkl', "rb") as f:
    acf_450_dict = pickle.load(f)
with open(results_folder + 'tau_isttc_full_50ms_20lags_len_450sec_dict.pkl', "rb") as f:
    sttc_450_dict = pickle.load(f)

# 600 sec
with open(results_folder + 'tau_acf_full_50ms_20lags_dict.pkl', "rb") as f:
    acf_600_dict = pickle.load(f)
with open(results_folder + 'tau_isttc_full_50ms_20lags_dict.pkl', "rb") as f:
    sttc_600_dict = pickle.load(f)

print(f'len acf_60_dict {len(acf_60_dict)}')
print(f'len sttc_60_dict {len(sttc_60_dict)}')

print(f'len acf_150_dict {len(acf_150_dict)}')
print(f'len sttc_150_dict {len(sttc_150_dict)}')

print(f'len acf_300_dict {len(acf_300_dict)}')
print(f'len sttc_300_dict {len(sttc_300_dict)}')

print(f'len acf_450_dict {len(acf_450_dict)}')
print(f'len sttc_450_dict {len(sttc_450_dict)}')

print(f'len acf_600_dict {len(acf_600_dict)}')
print(f'len sttc_600_dict {len(sttc_600_dict)}')

In [ ]:
acf_60_df = get_tau_df(acf_60_dict, 60, 'acf_full', alphas, fr_values, taus_ms)
acf_150_df = get_tau_df(acf_150_dict, 150, 'acf_full', alphas, fr_values, taus_ms)
acf_300_df = get_tau_df(acf_300_dict, 300, 'acf_full', alphas, fr_values, taus_ms)
acf_450_df = get_tau_df(acf_450_dict, 450, 'acf_full', alphas, fr_values, taus_ms)
acf_600_df = get_tau_df(acf_600_dict, 600, 'acf_full', alphas, fr_values, taus_ms)

isttc_60_df = get_tau_df(sttc_60_dict, 60, 'isttc_full', alphas, fr_values, taus_ms)
isttc_150_df = get_tau_df(sttc_150_dict, 150, 'isttc_full', alphas, fr_values, taus_ms)
isttc_300_df = get_tau_df(sttc_300_dict, 300, 'isttc_full', alphas, fr_values, taus_ms)
isttc_450_df = get_tau_df(sttc_450_dict, 450, 'isttc_full', alphas, fr_values, taus_ms)
isttc_600_df = get_tau_df(sttc_600_dict, 600, 'isttc_full', alphas, fr_values, taus_ms)

In [ ]:
def clean_pair(acf_df, isttc_df, id_col="unit_id", r2_col="fit_r_squared", label=None):
    acf_nans_units   = set(acf_df.loc[acf_df.isna().any(axis=1), id_col])
    isttc_nans_units = set(isttc_df.loc[isttc_df.isna().any(axis=1), id_col])
    neg_acf_units    = set(acf_df.loc[acf_df[r2_col] < 0, id_col])
    neg_isttc_units  = set(isttc_df.loc[isttc_df[r2_col] < 0, id_col])

    drop_units = acf_nans_units | isttc_nans_units | neg_acf_units | neg_isttc_units

    acf_clean   = acf_df[~acf_df[id_col].isin(drop_units)].copy()
    isttc_clean = isttc_df[~isttc_df[id_col].isin(drop_units)].copy()

    stats = {
        "label": label,
        "acf_nans": len(acf_nans_units),
        "isttc_nans": len(isttc_nans_units),
        "neg_acf": len(neg_acf_units),
        "neg_isttc": len(neg_isttc_units),
        "dropped_units_total": len(drop_units),
        "acf_len": len(acf_clean),
        "isttc_len": len(isttc_clean),
    }
    return acf_clean, isttc_clean, stats

pairs = {
    60:  (acf_60_df,  isttc_60_df),
    150: (acf_150_df, isttc_150_df),
    300: (acf_300_df, isttc_300_df),
    450: (acf_450_df, isttc_450_df),
    600: (acf_600_df, isttc_600_df),
}

cleaned = {}
summary_rows = []
for win, (acf_df, ist_df) in pairs.items():
    acf_c, ist_c, stats = clean_pair(acf_df, ist_df, label=f"{win} ms")
    cleaned[win] = (acf_c, ist_c)
    summary_rows.append(stats)

# sanity check
summary_df = pd.DataFrame(summary_rows).set_index("label")
print(summary_df)

tau_full_long_var_length_df_clean = pd.concat(
    [df for win in sorted(cleaned) for df in cleaned[win]],
    ignore_index=True
)
print(f"len tau_full_long_var_length_df_clean {len(tau_full_long_var_length_df_clean)}")

In [ ]:
#save df
tau_full_long_var_length_df_clean.to_csv(results_folder + 'summary_tau_full_long_var_length_df.csv')
tau_full_long_var_length_df_clean.to_pickle(results_folder + 'summary_tau_full_long_var_length_df.pkl')

In [ ]:
tau_full_long_var_length_df = pd.concat(
    [df for _, (acf_df, isttc_df) in sorted(pairs.items()) for df in (acf_df, isttc_df)],
    ignore_index=True
)
print(f"len tau_full_long_var_length_df {len(tau_full_long_var_length_df)}")

#save df
tau_full_long_var_length_df.to_csv(results_folder + 'summary_tau_full_long_var_length_df_all_units.csv')
tau_full_long_var_length_df.to_pickle(results_folder + 'summary_tau_full_long_var_length_df_all_units.pkl')

### PearsonR vs iSTTC (concat), varying number of trials (40, 60, 80, 100)

In [ ]:
# 40 trials
with open(results_folder + 'tau_pearsonr_trial_50ms_20lags_dict.pkl', "rb") as f:
    pearsonr_trial_avg_40_dict = pickle.load(f)
with open(results_folder + 'tau_isttc_trial_concat_50ms_20lags_dict.pkl', "rb") as f:
    sttc_trial_concat_40_dict = pickle.load(f)

# 60 trials
with open(results_folder + 'tau_pearsonr_trial_50ms_20_lags_60_trials_dict.pkl', "rb") as f:
    pearsonr_trial_avg_60_dict = pickle.load(f)
with open(results_folder + 'tau_isttc_trial_concat_50ms_20lags_60_trials_dict.pkl', "rb") as f:
    sttc_trial_concat_60_dict = pickle.load(f)

# 80 trials
with open(results_folder + 'tau_pearsonr_trial_50ms_20_lags_80_trials_dict.pkl', "rb") as f:
    pearsonr_trial_avg_80_dict = pickle.load(f)
with open(results_folder + 'tau_isttc_trial_concat_50ms_20lags_80_trials_dict.pkl', "rb") as f:
    sttc_trial_concat_80_dict = pickle.load(f)

# 100 trials
with open(results_folder + 'tau_pearsonr_trial_50ms_20_lags_100_trials_dict.pkl', "rb") as f:
    pearsonr_trial_avg_100_dict = pickle.load(f)
with open(results_folder + 'tau_isttc_trial_concat_50ms_20lags_100_trials_dict.pkl', "rb") as f:
    sttc_trial_concat_100_dict = pickle.load(f)

print(f'len pearsonr_trial_avg_40_dict {len(pearsonr_trial_avg_40_dict)}')
print(f'len sttc_trial_concat_40_dict {len(sttc_trial_concat_40_dict)}')

print(f'len pearsonr_trial_avg_60_dict {len(pearsonr_trial_avg_60_dict)}')
print(f'len sttc_trial_concat_60_dict {len(sttc_trial_concat_60_dict)}')

print(f'len pearsonr_trial_avg_80_dict {len(pearsonr_trial_avg_80_dict)}')
print(f'len sttc_trial_concat_80_dict {len(sttc_trial_concat_80_dict)}')

print(f'len pearsonr_trial_avg_100_dict {len(pearsonr_trial_avg_100_dict)}')
print(f'len sttc_trial_concat_100_dict {len(sttc_trial_concat_100_dict)}')

In [ ]:
# for trial based measures one realization of trials is taken
random_trials_impl = np.zeros(len(pearsonr_trial_avg_40_dict)).astype(int)

pearsontr_trial_avg_40_df = get_trials_plot_df(pearsonr_trial_avg_40_dict, 40, 'pearsonr_trial_avg', alphas, fr_values, taus_ms, random_trials_impl)
pearsontr_trial_avg_60_df = get_trials_plot_df(pearsonr_trial_avg_60_dict, 60, 'pearsonr_trial_avg', alphas, fr_values, taus_ms, random_trials_impl)
pearsontr_trial_avg_80_df = get_trials_plot_df(pearsonr_trial_avg_80_dict, 80, 'pearsonr_trial_avg', alphas, fr_values, taus_ms, random_trials_impl)
pearsontr_trial_avg_100_df = get_trials_plot_df(pearsonr_trial_avg_100_dict, 100, 'pearsonr_trial_avg', alphas, fr_values, taus_ms, random_trials_impl)

sttc_trial_concat_40_df = get_trials_plot_df(sttc_trial_concat_40_dict, 40, 'sttc_trial_concat', alphas, fr_values, taus_ms, random_trials_impl)
sttc_trial_concat_60_df = get_trials_plot_df(sttc_trial_concat_60_dict, 60, 'sttc_trial_concat', alphas, fr_values, taus_ms, random_trials_impl)
sttc_trial_concat_80_df = get_trials_plot_df(sttc_trial_concat_80_dict, 80, 'sttc_trial_concat', alphas, fr_values, taus_ms, random_trials_impl)
sttc_trial_concat_100_df = get_trials_plot_df(sttc_trial_concat_100_dict, 100, 'sttc_trial_concat', alphas, fr_values, taus_ms, random_trials_impl)

In [ ]:
pairs = {
    40:  (pearsontr_trial_avg_40_df,  sttc_trial_concat_40_df),
    60: (pearsontr_trial_avg_60_df, sttc_trial_concat_60_df),
    80: (pearsontr_trial_avg_80_df, sttc_trial_concat_80_df),
    100: (pearsontr_trial_avg_100_df, sttc_trial_concat_100_df),
}

cleaned = {}
summary_rows = []
for win, (acf_df, ist_df) in pairs.items():
    acf_c, ist_c, stats = clean_pair(acf_df, ist_df, label=f"{win} ms")
    cleaned[win] = (acf_c, ist_c)
    summary_rows.append(stats)

# sanity check
summary_df = pd.DataFrame(summary_rows).set_index("label")
print(summary_df)

tau_trials_long_var_num_trials_df_clean = pd.concat(
    [df for win in sorted(cleaned) for df in cleaned[win]],
    ignore_index=True
)
print(f"len tau_trials_long_var_num_trials_df_clean {len(tau_trials_long_var_num_trials_df_clean)}")

In [ ]:
#save df
tau_trials_long_var_num_trials_df_clean.to_csv(results_folder + 'summary_tau_trials_long_var_num_trials_df.csv')
tau_trials_long_var_num_trials_df_clean.to_pickle(results_folder + 'summary_tau_trials_long_var_num_trials_df.pkl')

In [ ]:
tau_trials_long_var_num_trials_df = pd.concat(
    [df for _, (acf_df, isttc_df) in sorted(pairs.items()) for df in (acf_df, isttc_df)],
    ignore_index=True
)
print(f"len tau_trials_long_var_num_trials_df {len(tau_trials_long_var_num_trials_df)}")

#save df
tau_trials_long_var_num_trials_df.to_csv(results_folder + 'tau_trials_long_var_num_trials_df_all_units.csv')
tau_trials_long_var_num_trials_df.to_pickle(results_folder + 'tau_trials_long_var_num_trials_df_all_units.pkl')

### ACF and iSTTC full signal, different number of lags

In [12]:
results_folder_lags = project_folder_path + 'results\\synthetic\\results\\bin_size_runs\\full_signal\\'

acf_configs = [
    {
        "path": results_folder,  # assumes this is defined earlier
        "filename": "tau_acf_full_50ms_20lags_dict.pkl",
        "n_lags": 20,
        "bin_size": 50,       
    },
    {
        "path": results_folder_lags,
        "filename": "acf_full_100ms_10lags_dict.pkl",
        "n_lags": 10,
        "bin_size": 100,
    },
    {
        "path": results_folder_lags,
        "filename": "acf_full_10ms_100lags_dict.pkl",
        "n_lags": 100,
        "bin_size": 10,
    },
    {
        "path": results_folder_lags,
        "filename": "acf_full_40ms_25lags_dict.pkl",
        "n_lags": 25,
        "bin_size": 40,
    },
    {
        "path": results_folder_lags,
        "filename": "acf_full_60ms_16lags_dict.pkl",
        "n_lags": 16,
        "bin_size": 60,
    },
]

tau_dfs = []
for cfg in acf_configs:
    full_path = cfg["path"] + cfg["filename"]
    with open(full_path, "rb") as f:
        acf_dict = pickle.load(f)

    bin_size = cfg["bin_size"]
    print(f'len acf_full_dict {bin_size}ms {len(acf_dict)}')
    tau_df = get_tau_df(acf_dict, 600, 'acf_full', alphas, fr_values, taus_ms, bin_size)
    tau_df["n_lags"] = cfg["n_lags"]
    tau_df["bin_size"] = bin_size
    tau_dfs.append(tau_df)

# make long df
tau_full_long_df = pd.concat(tau_dfs, ignore_index=True)
tau_full_long_df = tau_full_long_df.merge(lv_df[["unit_id", "lv"]].copy(),
                                          on="unit_id", how="left")
print(f'len tau_full_long_df {len(tau_full_long_df)}')

# leave only units with both methods (no NaNs and r_squared >= 0)
required = {cfg["bin_size"] for cfg in acf_configs}
tau_full_long_df_clean = (
    tau_full_long_df.groupby("unit_id")
    .filter(
        lambda g: (
            len(g) == len(required)
            and set(g["bin_size"]) == required
            and g.notna().all().all()
            and (g["fit_r_squared"] >= 0).all()
        )
    )
    .reset_index(drop=True)
)

print(
    f'len tau_full_long_df_clean {len(tau_full_long_df_clean)}, '
    f'per method {len(tau_full_long_df_clean) / len(required)}'
)

# save df
tau_full_long_df_clean.to_csv(results_folder_lags + 'summary_tau_full_long_acf_lags_df.csv')
tau_full_long_df_clean.to_pickle(results_folder_lags + 'summary_tau_full_long_acf_lags_df.pkl')

tau_full_long_df.to_csv(results_folder_lags + 'summary_tau_full_long_acf_lags_df_all_units.csv')
tau_full_long_df.to_pickle(results_folder_lags + 'summary_tau_full_long_acf_lags_df_all_units.pkl')

len acf_full_dict 50ms 100000
N rows with NaNs 11
len acf_full_dict 100ms 100000
N rows with NaNs 11
len acf_full_dict 10ms 100000
N rows with NaNs 11
len acf_full_dict 40ms 100000
N rows with NaNs 11
len acf_full_dict 60ms 100000
N rows with NaNs 11
len tau_full_long_df 500000
len tau_full_long_df_clean 492235, per method 98447.0


In [13]:
results_folder_lags = project_folder_path + 'results\\synthetic\\results\\bin_size_runs\\full_signal\\'

acf_configs = [
    {
        "path": results_folder,  # assumes this is defined earlier
        "filename": "tau_isttc_full_50ms_20lags_dict.pkl",
        "n_lags": 20,
        "bin_size": 50,       
    },
    {
        "path": results_folder_lags,
        "filename": "acf_isttc_full_100ms_10lags_dict.pkl",
        "n_lags": 10,
        "bin_size": 100,
    },
    {
        "path": results_folder_lags,
        "filename": "acf_isttc_full_10ms_100lags_dict.pkl",
        "n_lags": 100,
        "bin_size": 10,
    },
    {
        "path": results_folder_lags,
        "filename": "acf_isttc_full_40ms_25lags_dict.pkl",
        "n_lags": 25,
        "bin_size": 40,
    },
    {
        "path": results_folder_lags,
        "filename": "acf_isttc_full_60ms_16lags_dict.pkl",
        "n_lags": 16,
        "bin_size": 60,
    },
]

tau_dfs = []
for cfg in acf_configs:
    full_path = cfg["path"] + cfg["filename"]
    with open(full_path, "rb") as f:
        acf_dict = pickle.load(f)

    bin_size = cfg["bin_size"]
    print(f'len acf_full_dict {bin_size}ms {len(acf_dict)}')
    tau_df = get_tau_df(acf_dict, 600, 'acf_isttc_full', alphas, fr_values, taus_ms, bin_size)
    tau_df["n_lags"] = cfg["n_lags"]
    tau_df["bin_size"] = bin_size
    tau_dfs.append(tau_df)

# make long df
tau_full_long_df = pd.concat(tau_dfs, ignore_index=True)
tau_full_long_df = tau_full_long_df.merge(lv_df[["unit_id", "lv"]].copy(),
                                          on="unit_id", how="left")
print(f'len tau_full_long_df {len(tau_full_long_df)}')

# leave only units with both methods (no NaNs and r_squared >= 0)
required = {cfg["bin_size"] for cfg in acf_configs}
tau_full_long_df_clean = (
    tau_full_long_df.groupby("unit_id")
    .filter(
        lambda g: (
            len(g) == len(required)
            and set(g["bin_size"]) == required
            and g.notna().all().all()
            and (g["fit_r_squared"] >= 0).all()
        )
    )
    .reset_index(drop=True)
)

print(
    f'len tau_full_long_df_clean {len(tau_full_long_df_clean)}, '
    f'per method {len(tau_full_long_df_clean) / len(required)}'
)

# save df
tau_full_long_df_clean.to_csv(results_folder_lags + 'summary_tau_full_long_acf_isttc_lags_df.csv')
tau_full_long_df_clean.to_pickle(results_folder_lags + 'summary_tau_full_long_acf_isttc_lags_df.pkl')

tau_full_long_df.to_csv(results_folder_lags + 'summary_tau_full_long_acf_isttc_lags_df_all_units.csv')
tau_full_long_df.to_pickle(results_folder_lags + 'summary_tau_full_long_acf_isttc_lags_df_all_units.pkl')

len acf_full_dict 50ms 100000
N rows with NaNs 0
len acf_full_dict 100ms 100000
N rows with NaNs 0
len acf_full_dict 10ms 100000
N rows with NaNs 0
len acf_full_dict 40ms 100000
N rows with NaNs 0
len acf_full_dict 60ms 100000
N rows with NaNs 0
len tau_full_long_df 500000
len tau_full_long_df_clean 492830, per method 98566.0
